In [4]:
import ee
from cloud_mask import maskS2clouds
from config import startDate, endDate, region, cloud_cover, Map
from Indices import NDVI_img, EVI_img, LSWI_img, SRWI_img, SRTI_img, NDTI_img, CRCI_img, MCRC_img, SAVI_img, NDSVI_img
from Date_format import format_date
from merge_S2_MODIS import merge_images
import pandas as pd
from meta_data_to_pandas import extract_metadata
from modis_projection import reproject_s2_to_modis_10m
ee.Authenticate()
ee.Initialize()


c:\Users\varun\anaconda3\envs\myenv\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (



Successfully saved authorization token.


In [5]:
# Sentinel-2 preprocessing
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(startDate, endDate).filterBounds(region).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)).map(maskS2clouds).map(NDVI_img).map(EVI_img).map(LSWI_img).map(SRTI_img).map(NDTI_img).map(CRCI_img).map(MCRC_img).map(SAVI_img).map(NDSVI_img).map(format_date)
# Example visualization
first_image = s2.first()
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}
#Map.addLayer(first_image, vis_params, 'Sentinel-2 RGB')
#Map.addLayer(region, {}, 'Region')
#Map


In [6]:
# MODIS preprocessing
modis = (ee.ImageCollection('MODIS/061/MCD43A3')
         .select('Albedo_WSA_shortwave')
         .filterDate(startDate, endDate)
         .map(format_date))

# modis_qa = ee.ImageCollection('MODIS/061/MCD43A2') \
#     .select('BRDF_Albedo_Band_Quality') \
#     .filterDate(startDate, endDate) \
#     .map(format_date)

# modis_clean = modis_qa.map(mask_modis_clouds)


In [7]:
# Join by date
join = ee.Join.inner()
date_filter = ee.Filter.equals(leftField='date', rightField='date')
joined = join.apply(s2, modis, date_filter)

In [8]:
mergedCollection = ee.ImageCollection(joined.map(merge_images))

# Debug print
#print(mergedCollection.getInfo())
Map.addLayer(mergedCollection.first(),{},"merged")
Map


Map(center=[-120.6189, 43.7314], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

# RUN IF YOU WANT TO EXPORT META DATA AS PANDAS DF

In [6]:
# Convert to FeatureCollection
metadata_fc = ee.FeatureCollection(mergedCollection.map(extract_metadata))

# Get data to client side
metadata_list = metadata_fc.getInfo()['features']

# Convert to pandas DataFrame
image_data = pd.DataFrame([f['properties'] for f in metadata_list])

# Optional: convert timestamp to datetime
image_data['system_time_start'] = pd.to_datetime(image_data['system_time_start'], unit='ms')

# Save to CSV if needed
image_data.to_csv("merged_image_metadata.csv", index=False)

# Print preview
print(image_data.head())

         date                                                 id  \
0  2021-01-13  20210113T183731_20210113T184009_T11TNK_2021_01_13   
1  2021-01-14  20210114T185729_20210114T185730_T10TFP_2021_01_14   
2  2021-01-14  20210114T185729_20210114T185730_T10TGN_2021_01_14   
3  2021-01-14  20210114T185729_20210114T185730_T10TGP_2021_01_14   
4  2021-01-14  20210114T185729_20210114T185730_T11TLG_2021_01_14   

        system_time_start  
0 2021-01-13 18:42:02.517  
1 2021-01-14 19:02:24.126  
2 2021-01-14 19:02:35.472  
3 2021-01-14 19:02:20.917  
4 2021-01-14 19:02:45.483  


# DOWNLOAD SINLE IMAGE

In [ ]:
# # DOWNLOAD FIRST IMAGE

# s2Image = mergedCollection.first().toFloat().select(['B.*', 'ndvi', 'evi', 'lswi', 'srti', 'ndti', 'crci', 'mcrc', 'savi', 'ndsvi'])
  
# modisImage = mergedCollection.first().toFloat().select('MODIS_Albedo_WSA_shortwave')

# # Get MODIS projection
# modisProj = modisImage.projection()

# # Reproject S2 to MODIS projection but keep 10m resolution
# s2_reprojected = reproject_s2_to_modis_10m(s2Image, modisProj)



# # Export Sentinel-2 reprojected image
# task1 = ee.batch.Export.image.toDrive(
#     image=s2_reprojected,
#     description='S2_10m_resolution_MODIS_CRS',
#     folder='EarthEngineExports',
#     fileNamePrefix='S2_10m_resolution_MODIS_CRS',
#     #region=region,
#     scale=10,
#     maxPixels=1e13,
#     fileFormat='GeoTIFF'
# )
# task1.start()


# # Export Sentinel-2 image (10m resolution, MODIS CRS)
# Export.image.toDrive({
#   image: s2_modis_crs_10m,
#   description: 'S2_10m_resolution_MODIS_CRS',
#   scale: 10,
#   maxPixels: 1e13
# });




# Export.image.toDrive({
#   image: modisImage,
#   description: 'MODIS_Albedo_resampled_500m',
#   scale: 500,
#   maxPixels: 1e13
# });



# task2 = ee.batch.Export.image.toDrive(
#     image=modisImage,
#     description='MODIS_Albedo_resampled_500m',
#     folder='EarthEngineExports',
#     fileNamePrefix='MODIS_Albedo_resampled_500m',
#     #region=region,
#     scale=500,
#     maxPixels=1e13,
#     fileFormat='GeoTIFF'
# )
# task2.start()

# print("Export tasks started.")
# print("Task 1 status:", task1.status())
# print("Task 2 status:", task2.status())

Export tasks started.
Task 1 status: {'state': 'READY', 'description': 'S2_10m_resolution_MODIS_CRS', 'creation_timestamp_ms': 1746643829378, 'update_timestamp_ms': 1746643829378, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'O3TBBPSTTYTT67PS77KTPSCX', 'name': 'projects/earthengine-legacy/operations/O3TBBPSTTYTT67PS77KTPSCX'}
Task 2 status: {'state': 'READY', 'description': 'MODIS_Albedo_resampled_500m', 'creation_timestamp_ms': 1746643829823, 'update_timestamp_ms': 1746643829823, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'OHCZTKQSYPOFEIF47YAGLHRQ', 'name': 'projects/earthengine-legacy/operations/OHCZTKQSYPOFEIF47YAGLHRQ'}


# DOWNLOAD BATCH IMAGES

In [ ]:
scale_s2 = 10
scale_modis = 500

# Function to reproject Sentinel-2 to MODIS CRS (define this elsewhere in your code)
def reproject_s2_to_modis_10m(s2_img, modis_proj):
    return s2_img.reproject(crs=modis_proj.crs(), scale=scale_s2)

# Get list of images in the collection
imageList = mergedCollection.toList(mergedCollection.size())


# Limit export count (optional)
export_count = 1  # set to mergedCollection.size().getInfo() to export all

for i in range(export_count):
    image = ee.Image(imageList.get(i))
    
    # Extract date
    date_str = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

    # Select S2 bands and reproject
    s2Image = image.toFloat().select(['B.*', 'ndvi', 'evi', 'lswi', 'srti', 'ndti', 'crci', 'mcrc', 'savi', 'ndsvi'])
    modisImage = image.toFloat().select('MODIS_Albedo_WSA_shortwave')
    
    modisProj = modisImage.projection()
    s2_reprojected = reproject_s2_to_modis_10m(s2Image, modisProj)

    # Start export for S2
    task1 = ee.batch.Export.image.toDrive(
        image=s2_reprojected,
        description=f'S2_{date_str}',
        folder='EarthEngineExports',
        fileNamePrefix=f'S2_{date_str}',
        #region=region,
        scale=scale_s2,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task1.start()

    # Start export for MODIS
    task2 = ee.batch.Export.image.toDrive(
        image=modisImage,
        description=f'MODIS_{date_str}',
        folder='EarthEngineExports',
        fileNamePrefix=f'MODIS_{date_str}',
        #region=region,
        scale=scale_modis,
        maxPixels=1e13,
        fileFormat='GeoTIFF'
    )
    task2.start()

    print(f"Started export for {date_str}")


Started export for 2021-01-13
